In [7]:
import requests
import pandas as pd
import xmltodict
from urllib.parse import quote
import time

apiKey = "5562477541746f62363870736a667a"
region_list = pd.read_excel("서울시 주요 116장소명 목록.xlsx")
result = {}

for area in region_list["AREA_NM"]:
    encoded_area = quote(area)  # URL 인코딩
    request_url = f"http://openapi.seoul.go.kr:8088/{apiKey}/xml/citydata_ppltn/1/5/{encoded_area}"
    req = requests.get(request_url)
    
    if req.status_code != 200:
        print(f"Failed to fetch data for {area}. Status code: {req.status_code}")
        continue
    
    try:
        data_dict = xmltodict.parse(req.text)  # XML을 딕셔너리로 변환
        result[area] = data_dict
    except Exception as e:
        print(f"Failed to parse XML for {area}. Error: {e}")

    time.sleep(0.5)

# 결과 확인
print(result)


C:\Users\Hello\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


{'강남 MICE 관광특구': {'Map': {'SeoulRtd.citydata_ppltn': {'AREA_NM': '강남 MICE 관광특구', 'AREA_CD': 'POI001', 'AREA_CONGEST_LVL': '여유', 'AREA_CONGEST_MSG': '사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.', 'AREA_PPLTN_MIN': '700', 'AREA_PPLTN_MAX': '800', 'MALE_PPLTN_RATE': '61.3', 'FEMALE_PPLTN_RATE': '38.7', 'PPLTN_RATE_0': '0.3', 'PPLTN_RATE_10': '1.9', 'PPLTN_RATE_20': '17.1', 'PPLTN_RATE_30': '22.9', 'PPLTN_RATE_40': '22.4', 'PPLTN_RATE_50': '16.2', 'PPLTN_RATE_60': '15.4', 'PPLTN_RATE_70': '3.8', 'RESNT_PPLTN_RATE': '20.6', 'NON_RESNT_PPLTN_RATE': '79.4', 'REPLACE_YN': 'N', 'PPLTN_TIME': '2025-01-12 06:10', 'FCST_YN': 'Y', 'FCST_PPLTN': {'FCST_PPLTN': [{'FCST_TIME': '2025-01-12 07:00', 'FCST_CONGEST_LVL': '여유', 'FCST_PPLTN_MIN': '600', 'FCST_PPLTN_MAX': '700'}, {'FCST_TIME': '2025-01-12 08:00', 'FCST_CONGEST_LVL': '여유', 'FCST_PPLTN_MIN': '1000', 'FCST_PPLTN_MAX': '1500'}, {'FCST_TIME': '2025-01-12 09:00', 'FCST_CONGEST_LVL': '여유', 'FCST_PPLTN_MIN': '2000', 'FCST_PPLTN_MAX': '2500'}, {'F

In [8]:
import json
with open("result.json", "w", encoding="utf-8") as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

In [9]:
import json
import pandas as pd

# JSON 파일 읽기
with open("result.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# 결과 저장을 위한 데이터프레임 초기화
columns = ["지역명"]  # 첫 번째 열은 지역명
population_data = []

for area, area_data in data.items():
    row = [area]  # 첫 번째 열에 지역명 추가

    # 현재 시간의 인구수 추정 (AREA_PPLTN_MIN과 AREA_PPLTN_MAX의 평균)
    try:
        current_min = int(area_data["Map"]["SeoulRtd.citydata_ppltn"]["AREA_PPLTN_MIN"])
        current_max = int(area_data["Map"]["SeoulRtd.citydata_ppltn"]["AREA_PPLTN_MAX"])
        current_population = (current_min + current_max) // 2
        row.append(current_population)
        if "06:00" not in columns:
            columns.append("06:00")
    except KeyError:
        row.append(None)  # 데이터가 없는 경우

    # 예측 시간대의 인구수 추정
    try:
        fcst_data = area_data["Map"]["SeoulRtd.citydata_ppltn"]["FCST_PPLTN"]["FCST_PPLTN"]
        for fcst in fcst_data:
            fcst_time = fcst["FCST_TIME"][-5:]  # 시간 추출 (예: "07:00")
            fcst_min = int(fcst["FCST_PPLTN_MIN"])
            fcst_max = int(fcst["FCST_PPLTN_MAX"])
            fcst_population = (fcst_min + fcst_max) // 2
            row.append(fcst_population)

            if fcst_time not in columns:
                columns.append(fcst_time)
    except KeyError:
        pass

    population_data.append(row)

# 데이터프레임 생성
population_df = pd.DataFrame(population_data, columns=columns)

# NaN 값을 0으로 채우기 (옵션)
population_df = population_df.fillna(0)

# 데이터프레임 저장
population_df.to_excel("지역별_시간별_인구수.xlsx", index=False)
print("결과가 지역별_시간별_인구수.xlsx 파일로 저장되었습니다.")


결과가 지역별_시간별_인구수.xlsx 파일로 저장되었습니다.


In [11]:
population_df.drop("06:00", axis = 1, inplace = True)

In [12]:
population_df

,지역명,07:00,08:00,09:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00
0,강남 MICE 관광특구,650,1250,2250,4250,6750,11000,13000,15000,17000,17000,17000,17000
1,동대문 관광특구,5750,6750,7750,8750,11000,13000,17000,19000,19000,19000,19000,19000
2,명동 관광특구,5750,7250,9250,13000,17000,23000,29000,33000,35000,35000,33000,31000
3,이태원 관광특구,5250,4250,3750,4250,4750,5750,6750,7750,8750,9750,9750,9750
4,잠실 관광특구,15000,17000,21000,25000,33000,43000,53000,63000,69000,71000,73000,73000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,청계산,450,550,650,750,850,850,850,850,750,750,650,750
112,청와대,0,150,150,250,350,350,250,250,350,350,250,150
113,북창동 먹자골목,750,850,1250,1250,1750,2250,2750,3250,3250,3250,3250,2750
114,남대문시장,650,750,1250,1750,2250,3750,4750,6250,6750,7250,6750,5750


In [14]:
import numpy as np

population_data = pd.read_excel("지역별_시간별_인구수.xlsx")
# 포아송 분포를 이용한 대기 시간 생성 함수
def generate_wait_times(population_density):
    # 인구 밀도가 높을수록 평균 대기 시간을 줄임
    max_density = population_density.max()
    scaled_density = max_density - population_density  # 높은 밀도 -> 낮은 대기 시간
    mean_wait_times = scaled_density / max_density * 10  # 평균 대기 시간: 0 ~ 10분
    return np.random.poisson(mean_wait_times)

# 데이터 준비
wait_times_data = population_data.copy()
time_columns = wait_times_data.columns[1:]  # 시간대 컬럼

# 각 시간대에 대해 대기 시간 생성
for time_col in time_columns:
    wait_times_data[time_col] = generate_wait_times(wait_times_data[time_col])

# 결과 확인
wait_times_data.head()

,지역명,06:00,07:00,08:00,09:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00
0,강남 MICE 관광특구,8,10,12,11,12,5,9,3,5,8,2,7,10
1,동대문 관광특구,8,9,13,5,9,3,9,8,1,8,9,11,7
2,명동 관광특구,10,5,9,11,13,11,7,4,4,8,8,4,6
3,이태원 관광특구,9,10,10,8,4,8,5,5,14,14,5,11,8
4,잠실 관광특구,4,7,12,5,2,4,1,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 데이터 준비
# 기준 금액 833원에서 대기 시간과 인구 밀도를 활용한 금액 계산
base_price = 833

# 금액 계산 함수
def calculate_price(population_density, wait_time):
    population_factor = population_density / population_density.max()  # 인구 밀도 비율
    wait_time_factor = wait_time / wait_time.max()  # 대기 시간 비율
    price = base_price + 100 * (1 - population_factor) + 50 * wait_time_factor  # 간단한 조정
    return price

# 금액 데이터 생성
price_data = population_data.copy()
for time_col in time_columns:
    price_data[time_col] = calculate_price(
        population_data[time_col], wait_times_data[time_col]
    )

# 학습 데이터 생성 (시간대별로 피처와 타깃 값 생성)
features = []
targets = []
for time_col in time_columns:
    for i in range(len(population_data)):
        features.append([population_data.loc[i, time_col], wait_times_data.loc[i, time_col]])
        targets.append(price_data.loc[i, time_col])

# 데이터를 numpy 배열로 변환
X = np.array(features)  # 인구 밀도와 대기 시간을 피처로 사용
y = np.array(targets)   # 금액을 타깃 값으로 사용

# 데이터 분할 (학습용과 테스트용)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# 결과 출력
print("모델 평가 (MSE):", mse)
print("모델 계수 (인구 밀도 영향):", model.coef_[0])
print("모델 계수 (대기 시간 영향):", model.coef_[1])


모델 평가 (MSE): 32.338100391389695
모델 계수 (인구 밀도 영향): -0.0015703218615059043
모델 계수 (대기 시간 영향): 3.0951163170505374


In [23]:
import firebase_admin
from firebase_admin import credentials, firestore

# Firebase 초기화
if not firebase_admin._apps:  # Firebase 앱이 초기화되지 않은 경우만 초기화
    cred = credentials.Certificate("gbike-management-sys-firebase-adminsdk-l6el6-b5f711a993.json")  # 서비스 계정 키 파일 경로
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Firestore에 데이터 업로드 함수
def upload_to_firestore(data, wait_times, prices):
    for index, row in data.iterrows():
        area_name = row['지역명']  # 지역명
        area_data = {}  # Firestore 문서 데이터

        for time in row.index[1:]:
            area_data[time] = {
                "population": int(row[time]),  # numpy 타입 -> Python int로 변환
                "wait_time": int(wait_times.loc[index, time]),  # numpy 타입 -> Python int로 변환
                "price": float(prices.loc[index, time])  # numpy 타입 -> Python float로 변환
            }

        # Firestore에 업로드
        db.collection("지역별_킥보드_정보").document(area_name).set(area_data)
        print(f"{area_name} 데이터를 업로드했습니다.")


# Firestore에 데이터 업로드 실행
upload_to_firestore(population_data, wait_times_data, price_data)

강남 MICE 관광특구 데이터를 업로드했습니다.
동대문 관광특구 데이터를 업로드했습니다.
명동 관광특구 데이터를 업로드했습니다.
이태원 관광특구 데이터를 업로드했습니다.
잠실 관광특구 데이터를 업로드했습니다.
종로·청계 관광특구 데이터를 업로드했습니다.
홍대 관광특구 데이터를 업로드했습니다.
경복궁 데이터를 업로드했습니다.
광화문·덕수궁 데이터를 업로드했습니다.
보신각 데이터를 업로드했습니다.
서울 암사동 유적 데이터를 업로드했습니다.
창덕궁·종묘 데이터를 업로드했습니다.
가산디지털단지역 데이터를 업로드했습니다.
강남역 데이터를 업로드했습니다.
건대입구역 데이터를 업로드했습니다.
고덕역 데이터를 업로드했습니다.
고속터미널역 데이터를 업로드했습니다.
교대역 데이터를 업로드했습니다.
구로디지털단지역 데이터를 업로드했습니다.
구로역 데이터를 업로드했습니다.
군자역 데이터를 업로드했습니다.
남구로역 데이터를 업로드했습니다.
대림역 데이터를 업로드했습니다.
동대문역 데이터를 업로드했습니다.
뚝섬역 데이터를 업로드했습니다.
미아사거리역 데이터를 업로드했습니다.
발산역 데이터를 업로드했습니다.
북한산우이역 데이터를 업로드했습니다.
사당역 데이터를 업로드했습니다.
삼각지역 데이터를 업로드했습니다.
서울대입구역 데이터를 업로드했습니다.
서울식물원·마곡나루역 데이터를 업로드했습니다.
서울역 데이터를 업로드했습니다.
선릉역 데이터를 업로드했습니다.
성신여대입구역 데이터를 업로드했습니다.
수유역 데이터를 업로드했습니다.
신논현역·논현역 데이터를 업로드했습니다.
신도림역 데이터를 업로드했습니다.
신림역 데이터를 업로드했습니다.
신촌·이대역 데이터를 업로드했습니다.
양재역 데이터를 업로드했습니다.
역삼역 데이터를 업로드했습니다.
연신내역 데이터를 업로드했습니다.
오목교역·목동운동장 데이터를 업로드했습니다.
왕십리역 데이터를 업로드했습니다.
용산역 데이터를 업로드했습니다.
이태원역 데이터를 업로드했습니다.
장지역 데이터를 업로드했습니다.
장한평역 데이터를 업로드했습니다.
천호역 데이터를